In [1]:
import json
from utils.mongo import get_src_db
from pyld import jsonld
import os.path

In [2]:
def get_doc_from_mongo(hgvs_id):
    '''give an id, return the doc corresponding to that id in mongodb'''
    src = get_src_db()
    doc = src.merge_201511.find({'_id': hgvs_id}).next()
    return doc

In [3]:
def load_context(context_file_path):
    context = json.loads(open(context_file_path).read())
    return context

In [4]:
def find_doc(k,keys):
    n = len(keys)
    for i in range(n):
        # if k is a dictionary, then directly get its value
        if type(k) == dict:
            k = k[keys[i]]

        # if k is a list, then loop through k
        elif type(k) == list:
            tmp = []
            for item in k:
                try:
                    if type(item[keys[i]]) == dict:
                        tmp.append(item[keys[i]])
                    elif type(item[keys[i]]) == list:
                        for _item in item[keys[i]]:
                            tmp.append(_item)
                except:
                    continue
            k = tmp
    return k

In [5]:
 def add_context(doc, context):
    '''add jsonld context to the document'''
    for key in context:
        keys = key.split('/')
        try:
            doc_new = find_doc(doc,keys)
            if type(doc_new) == list:
                for _d in doc_new:
                    _d.update(context[key])
            elif type(doc_new) == dict:
                doc_new.update(context[key])
            else:
                print error
        except:
            continue
            print 'keyerror'
    return doc

In [6]:
def nquads_transform(doc):
    t = jsonld.JsonLdProcessor()
    nquads = t.parse_nquads(jsonld.to_rdf(doc, {'format': 'application/nquads'}))['@default']
    return nquads

In [23]:
def fetch_value_source_for_variant(id, context_path, uri):
    doc = get_doc_from_mongo(id)
    context = load_context(context_path)
    doc.update(context.pop('root'))
    doc = add_context(doc,context)
    nquads = nquads_transform(doc)
    (node, value) = get_value_and_node(nquads,uri)
    source = []
    for item in node:
        source.append(find_top_level_uri(item, nquads))
    result = [i + ' ' + j for i,j in zip(value, source)]
    return result
    

In [24]:
def get_value_and_node(nquads, uri):
    node_list = []
    value_list = []

    for item in nquads:
        if item['predicate']['value'] == uri:
            node_list.append(item['subject']['value'])
            value_list.append(item['object']['value'])
    return (node_list, value_list)

In [25]:
def find_top_level_uri(nquads_id, nquads):
    for item in nquads:
        if item['object']['value'] == nquads_id:
            if item['predicate']['value'] in top_level_uris:
                uri = item['predicate']['value']
            elif item['predicate']['value'] not in top_level_uris:
                uri = find_top_level_uri(item['subject']['value'], nquads)
            else:
                print "couldn't find top level uri"
    return uri

In [26]:
top_level_uris = ['http://example.org/cadd/', 
                  'http://example.org/clinvar/', 
                  'http://example.org/dbnsfp/', 
                  'http://example.org/dbsnp/', 
                  'http://example.org/docm/', 
                  'http://example.org/emv/', 
                  'http://example.org/evs/', 
                  'http://example.org/gwassnps/', 
                  'http://example.org/snpeff/', 
                  'http://example.org/mutdb/']

In [29]:
hgvs_id = 'chr6:g.26093141G>A'

In [30]:
src_path = os.path.split(os.path.split(os.path.abspath(__file__))[0])[0]
path = os.path.join(src_path, 'www/context/context.json')

In [31]:
uri = "http://identifiers.org/ensembl/"

In [32]:
fetch_value_source_for_variant(hgvs_id, path, uri)

[u'ENST00000357618 http://example.org/cadd/']

get ensembl id info

In [33]:
uri = "http://identifiers.org/ensembl"

In [34]:
fetch_value_source_for_variant(hgvs_id, path, uri)

[u'ENSG00000010704 http://example.org/cadd/',
 u'ENSG00000010704 http://example.org/dbnsfp/',
 u'ENSP00000259699 http://example.org/dbnsfp/',
 u'ENSP00000311698 http://example.org/dbnsfp/',
 u'ENSP00000312342 http://example.org/dbnsfp/',
 u'ENSP00000313776 http://example.org/dbnsfp/',
 u'ENSP00000337819 http://example.org/dbnsfp/',
 u'ENSP00000380217 http://example.org/dbnsfp/',
 u'ENSP00000417404 http://example.org/dbnsfp/',
 u'ENSP00000419725 http://example.org/dbnsfp/',
 u'ENSP00000420559 http://example.org/dbnsfp/',
 u'ENSP00000420802 http://example.org/dbnsfp/']

Get gene symbol info

In [35]:
uri = "http://identifiers.org/hgnc.symbol/"

In [36]:
fetch_value_source_for_variant(hgvs_id, path, uri)

[u'HFE http://example.org/cadd/',
 u'HFE http://example.org/dbnsfp/',
 u'HFE http://example.org/clinvar/',
 u'HFE http://example.org/dbsnp/',
 u'HFE http://example.org/evs/',
 u'HFE http://example.org/evs/',
 u'HFE http://example.org/evs/',
 u'HFE http://example.org/evs/',
 u'HFE http://example.org/evs/',
 u'HFE http://example.org/evs/',
 u'HFE http://example.org/evs/',
 u'HFE http://example.org/evs/',
 u'HFE http://example.org/evs/',
 u'HFE http://example.org/gwassnps/']

Get omim id info

In [37]:
uri = "http://identifiers.org/omim/"

In [38]:
fetch_value_source_for_variant(hgvs_id, path, uri)

[u'614193 http://example.org/clinvar/',
 u'612635 http://example.org/clinvar/',
 u'613609.0001 http://example.org/clinvar/',
 u'235200 http://example.org/clinvar/']

Get rsid info

In [39]:
uri = "http://identifiers.org/dbsnp/"

In [40]:
fetch_value_source_for_variant(hgvs_id, path, uri)

[u'rs1800562 http://example.org/dbnsfp/',
 u'rs1800562 http://example.org/clinvar/',
 u'rs1800562 http://example.org/dbsnp/',
 u'rs1800562 http://example.org/evs/',
 u'rs1800562 http://example.org/gwassnps/',
 u'rs1800562 http://example.org/mutdb/']

In [42]:
doc = get_doc_from_mongo(hgvs_id)

In [43]:
context = load_context(path)

In [44]:
    doc.update(context.pop('root'))
    doc = add_context(doc,context)

In [45]:
t = jsonld.JsonLdProcessor()

In [46]:
nquads = t.parse_nquads(jsonld.to_rdf(doc, {'format': 'application/nquads'}))

In [47]:
nquads

{'@default': [{'object': {'type': 'blank node', 'value': u'_:b1'},
   'predicate': {'type': 'IRI', 'value': u'http://example.org/cadd/'},
   'subject': {'type': 'blank node', 'value': u'_:b0'}},
  {'object': {'type': 'blank node', 'value': u'_:b3'},
   'predicate': {'type': 'IRI', 'value': u'http://example.org/clinvar/'},
   'subject': {'type': 'blank node', 'value': u'_:b0'}},
  {'object': {'type': 'blank node', 'value': u'_:b28'},
   'predicate': {'type': 'IRI', 'value': u'http://example.org/dbnsfp/'},
   'subject': {'type': 'blank node', 'value': u'_:b0'}},
  {'object': {'type': 'blank node', 'value': u'_:b40'},
   'predicate': {'type': 'IRI', 'value': u'http://example.org/dbsnp/'},
   'subject': {'type': 'blank node', 'value': u'_:b0'}},
  {'object': {'type': 'blank node', 'value': u'_:b42'},
   'predicate': {'type': 'IRI', 'value': u'http://example.org/evs/'},
   'subject': {'type': 'blank node', 'value': u'_:b0'}},
  {'object': {'type': 'blank node', 'value': u'_:b52'},
   'predi